## Principal Components Analysis (PCA)

PCA is another term for SVD when it is used for dimension reduction, to which LSA is in part constructed from.
<br>
The PCA model in `sklearn` contains some tweaks to the **SVD** math that'll improve the accuracy of a NLP pipeline.
<br>
* operations - `sklearn.PCA` automatically ***centres*** the data by subtracting off the mean word frequencies. Additionally, a more subtle trick is that PCA uses a function known as `flip_sign` to compute the sign of the singular vectors
* optional - The `sklearn` implementation of PCA implements an optional 'whitening' step. This is also similar to the trick of ignoring the singular values when transforming word-document vectors into topic-document vectors. Also, instead of just setting all the singular values is $S$ to one, whitening divides the data by the variances similar to how `sklearn.StandardScalar` transform does. This helps spread out the data and make any optimisation algorithm less likely to get get confused parts of the data that can arise when features in our dataset are correlated with each other 

PCA is useful for a wide range of applications, which is helpful to know as it can perform data science techniques beyond NLP.
<br>
For most applied problems, we'll want to use the `sklearn.PCA` model for a LSA model.

As an initial example, We're going to begin with a set of real-world 3D vectors rather than +10k dimensional document-word vectors - given it's easier to visualise things in 3D than in 10000-D.

### PCA on 3D vectors 